In [2]:
import os
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from openai import OpenAI


  # It will automatically pick up the env var
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            # "api_type": "azure",
            "api_key": OPENAI_API_KEY
            # "base_url": "https://hw2aiservices.openai.azure.com/",
            # "api_version": "2024-05-13",
        }
    ]
}

def get_last_reply_from(cur_agent):
    # get cur_agent's last reply to others
    for msg_list in cur_agent.chat_messages.values():
        for msg in reversed(msg_list):  # 从后往前找
            if msg.get("role") == "assistant" and msg.get("name") == cur_agent.name:
                return msg["content"]
    return None

def print_agent_chat_history(agent):
    print(f"\n--- Chat history for agent: {agent.name} ---")
    for msg in agent.chat_messages:
        # 安全提取 sender
        print(msg)
        # sender_raw = msg.get("name", "Unknown")
        # sender = sender_raw.name if hasattr(sender_raw, "name") else str(sender_raw)
        
        # content = msg.get("content", "")
        # print(f"{sender} ➜ {agent.name}: {content}\n")

class SpokespersonAgent(AssistantAgent):
    pass

class RecommenderAgent(AssistantAgent):
    pass

class CompletenessCheckAgent(AssistantAgent):
    pass

In [3]:
spokesperson_agent = SpokespersonAgent(
    name="spokesperson_agent",
    llm_config=llm_config,
    system_message="""
    You are the spokesperson of the research team. Your job is to assess the trader's decision and evaluate whether it sufficiently considers the team's prior Bullish and Bearish arguments.
    
    If you think the trader's response reflects the team's views, say:
    "TERMINATE. Your decision reflects our team's analysis. You may stop here."

    Otherwise, ask for clarification or adjustment.
    """
)

In [6]:
spokesperson_agent = SpokespersonAgent(
    name="spokesperson_agent",
    llm_config=llm_config,
    system_message="""
    You are the spokesperson of the research team. Your job is to assess the trader's decision and evaluate whether it sufficiently considers the team's prior Bullish and Bearish arguments.
    
    If you think the trader's response reflects the team's views, say:
    "TERMINATE. Your decision reflects our team's analysis. You may stop here."

    Otherwise, ask for clarification or adjustment.
    """
)

trade_recommender_agent = RecommenderAgent(
    name="trade_recommender_agent",
    llm_config=llm_config,
    system_message="""
    You are the delivering Trader's decision to risk team
    whatever they say, just reply:
    "TERMINATE"
    """
)

completeness_check_agent = CompletenessCheckAgent(
    name="trade_recommender_agent",
    llm_config=llm_config,
    system_message="""
    You are the checking manager_agent's decision
    if they give certain decision in EXECUTE_TRADE or DO_NOT_EXECUTE, reply:
    "FINISHED"
    """
)

class TraderAgentWithTermination(AssistantAgent):
    def is_termination_msg(self, message):
        # 只要包含 BUY 或 SELL 就终止
        content = message.get("content", "").lower()
        return "buy" in content or "sell" in content


# === Mock function for stock data collection ===
def data_collect(stock_name):
    return {
        "company_info": f"{stock_name} is a tech company founded in 2005.",
        "stock_price_history": [120, 123, 119, 125, 130],
        "social_media_sentiment": "Mixed, trending positive."
    }

# === Analyst Agent ===
analyst_agent = AssistantAgent(
    name="analyst_agent",
    llm_config=llm_config,
    system_message="""
    You are a market data analyst. You are allowed to call data_collect(stock_name) to gather information.
    Do NOT make investment decisions. Simply output the result of data_collect().
    """
)

# === Bullish Research Agent ===
bullish_agent = AssistantAgent(
    name="bullish_research_agent",
    llm_config=llm_config,
    system_message="""
    You are an optimistic stock researcher. Your job is to analyze data and argue why the stock is a good investment.
    You do not consider risks or negative sentiment. Your goal is to convince others the stock should be bought.
    """
)

# === Bearish Research Agent ===
bearish_agent = AssistantAgent(
    name="bearish_research_agent",
    llm_config=llm_config,
    system_message="""
    You are a conservative stock researcher. Your job is to analyze data and argue why the stock is risky or not a good investment.
    Your goal is to point out why the stock should be avoided.
    """
)

# === Trader Agent ===
trader_agent = AssistantAgent(
    name="trader_agent",
    llm_config=llm_config,
    system_message="""
    You are a trader. You read the bullish and bearish arguments and make a BUY or SELL suggestion.
    Do NOT consider risk preference.
    Just focus on the quality of the arguments and market outlook.
    Output your recommendation as: BUY or SELL and a short justification.
    """,
    is_termination_msg=lambda msg: "terminate" in msg.get("content", "").lower()
)


# === RRisk Management Team Agent ===
# risk_profile = os.getenv("RISK_PROFILE", "Neutral")
risk_profile = "Neutral"

risk_manager_agent = AssistantAgent(
    name="risk_manager_agent",
    llm_config=llm_config,
    system_message=f"""
    You are Risk Management Team. Current risk preference is: {risk_profile}.
    You evaluate trader's suggestion and tag it as Aggressive, Neutral, or Conservative.
    You can approve or reject the trade suggestion based on risk alignment.
    Only output one of these: APPROVED (with tag), or REJECTED (with reason).
    """,
    is_termination_msg=lambda msg: "terminate" in msg.get("content", "").lower()
)

# === Manager Agent ===
manager_agent = AssistantAgent(
    name="manager_agent",
    llm_config=llm_config,
    system_message="""
    You are the manager. You make the final decision to execute the trade.
    Your output should include EXECUTE_TRADE or DO_NOT_EXECUTE and give reasons.
    """,
    is_termination_msg=lambda msg: "finished" in msg.get("content", "").lower()
)

# === User Proxy Agent to initiate the workflow ===
user_proxy = UserProxyAgent(name="user_proxy_agent", human_input_mode="NEVER", code_execution_config={"use_docker": False})

# === Group chat for Bullish/Bearish debate ===
debate_group = GroupChat(
    agents=[bullish_agent, bearish_agent],
    messages=[],
    max_round=3,
    speaker_selection_method="round_robin"
)
debate_manager = GroupChatManager(groupchat=debate_group)

# === Main Orchestration Function ===
def run_stock_recommendation(stock_name):
    print("\n=== Step 1: Analyst collects data ===")
    stock_data = data_collect(stock_name)
    print("Data Collected:", stock_data)

    stock_data_str = (
        f"Stock Name: {stock_name}\n\n"
        f"Company Info: {stock_data['company_info']}\n"
        f"Stock Price History: {stock_data['stock_price_history']}\n"
        f"Social Media Sentiment: {stock_data['social_media_sentiment']}\n"
    )

    print("\n=== Step 2: Bullish vs Bearish debate ===")
    user_proxy.initiate_chat(debate_manager, message=f"The following stock data is available:\n{stock_data_str}")

    debate_summary = "\n--- Debate Summary ---\n"
    for msg in debate_group.messages:
        debate_summary += f"{msg['name']}: {msg['content']}\n"

    print("\n=== Step 3: Trader makes a decision ===")
    trader_prompt = (
        f"{stock_data_str}\n"
        f"{debate_summary}\n"
        "Based on the above, please make a BUY or SELL decision with reasoning.\n"
    )
    

    spokesperson_agent.initiate_chat(trader_agent, message=trader_prompt)

    trader_agent_response = get_last_reply_from(trader_agent)
    # trader_agent_response = user_proxy.initiate_chat(trader_agent, message=trader_prompt)

    # print("********************************")
    # print(trader_agent_response)
    # print("********************************")


    print("\n=== Step 4: Risk Management Team reviews ===")
    risk_prompt = (
        f"{stock_data_str}\n\n"
        f"{debate_summary}\n"
        f"Trader's Decision:\n{trader_agent_response}\n\n"
        f"Current Risk Profile: {risk_profile}\n"
        "Evaluate if this decision aligns with risk preferences. Approve or reject."
    )
    trade_recommender_agent.initiate_chat(risk_manager_agent, message=risk_prompt)
    

    risk_decision = get_last_reply_from(risk_manager_agent)

    print("\n=== Step 5: Manager final decision ===")
    manager_prompt = (
        f"{stock_data_str}\n\n"
        f"{debate_summary}\n"
        f"Trader's Decision:\n{trader_agent_response}\n\n"
        f"Risk Management Team's Decision:\n{risk_decision}\n\n"
        "Should we execute the trade?"
    )
    completeness_check_agent.initiate_chat(manager_agent, message=manager_prompt)


# === Example Usage ===
if __name__ == "__main__":
    run_stock_recommendation("Tesla")



=== Step 1: Analyst collects data ===
Data Collected: {'company_info': 'Tesla is a tech company founded in 2005.', 'stock_price_history': [120, 123, 119, 125, 130], 'social_media_sentiment': 'Mixed, trending positive.'}

=== Step 2: Bullish vs Bearish debate ===
user_proxy_agent (to chat_manager):

The following stock data is available:
Stock Name: Tesla

Company Info: Tesla is a tech company founded in 2005.
Stock Price History: [120, 123, 119, 125, 130]
Social Media Sentiment: Mixed, trending positive.


--------------------------------------------------------------------------------

Next speaker: bullish_research_agent

bullish_research_agent (to chat_manager):

Tesla, a pioneering force in the tech industry since 2005, offers a compelling investment opportunity with its innovative vision and strong market presence. Let's delve into why buying Tesla stock is a smart choice for those looking to invest in a remarkable growth story.

1. **Steady Stock Price Appreciation**: Examining 

In [ ]:
import yfinance as yf
data = yf.download("AAPL", start="2020-01-01", end="2023-12-31")

print(data)